In [63]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import ipywidgets as widgets
from IPython.display import display

In [64]:
df = pd.read_csv('classes_ok.csv', index_col= 0)
r_teach = pd.read_csv('teachers_grouped(str_group).csv')
w_teach = pd.merge(df, r_teach, how='left', on=['Дисциплина', 'Кафедра', 'Семестр', 'Группа'])
w_teach = w_teach.drop(['Курс_y'], axis=1)

In [65]:
w_teach.sort_values(by=['Дисциплина', 'Группа', 'фио'])

,НомерЛД,ПерваяПопытка,Семестр,Дисциплина,Кафедра,Тип_Ведомости,ID ЛК,Иностранец,Курс_x,finans,status,Уровень образования,Форма образования,Сезон,Год,visit,Группа,фио
2311,220320,4,2,Алгебра и геометрия,каф.ВМ,Экзамен,383979.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,spring,2024,0.000000,321,NaN
2319,220320,5,1,Алгебра и геометрия,каф.ВМ,Экзамен,383979.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,autumn,2023,24.448529,321,NaN
1133,120123,3,1,Алгебра и геометрия,каф.ВМ,Экзамен,417061.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,autumn,2023,72.794118,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
2278,220123,5,1,Алгебра и геометрия,каф.ВМ,Экзамен,416841.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,autumn,2023,78.860294,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
2318,220320,5,1,Алгебра и геометрия,каф.ВМ,Экзамен,383979.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,autumn,2023,24.448529,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5442,520623,5,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,418171.0,False,1.0,Контракт,Учащийся,Бакалавриат,Очная форма,spring,2024,56.250000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
6473,610623,4,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,417828.0,False,1.0,Контракт,Учащийся,Бакалавриат,Очная форма,spring,2024,43.750000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
8041,750623,5,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,417706.0,False,1.0,Контракт,Учащийся,Бакалавриат,Очная форма,spring,2024,68.750000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
3168,290623,5,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,418290.0,False,1.0,Бюджет,Учащийся,Бакалавриат,Очная форма,spring,2024,56.250000,3692,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."


In [66]:
your_dataframe = w_teach

# фильтруем
df_sorted = your_dataframe.sort_values(by=['Дисциплина', 'Группа', 'фио'])
df_sorted = df_sorted[(df_sorted['Семестр'] != 3)]
                      # & (df_sorted['Тип_Ведомости'] != 'К/проект')
                      # & (df_sorted['Дисциплина'] == 'Физика')]

# Разделяем на обучающую и тестовую выборку по четности ID ЛК
df1 = df_sorted[df_sorted['ID ЛК'] % 2 == 0].copy()
df2 = df_sorted[df_sorted['ID ЛК'] % 2 == 1].copy()

# Исключаем определенные записи
df1 = df1[df1['фио'] != 'КоцюбаИгорьЮрьевич, ГороховВладимирЛеонидович']

X_train = df1.drop(['ID ЛК', 'НомерЛД', 'ПерваяПопытка', 'Год', 'Уровень образования', 'Форма образования', 'Сезон', 'Курс_x'], axis=1).reset_index(drop=True)
y_train = df1['ПерваяПопытка'].reset_index(drop=True)

X_test = df2.drop(['ID ЛК', 'НомерЛД', 'ПерваяПопытка', 'Год', 'Уровень образования', 'Форма образования', 'Сезон', 'Курс_x'], axis=1).reset_index(drop=True)
y_test = df2['ПерваяПопытка'].reset_index(drop=True)

# Определим категориальные признаки
categorical_features = ['Кафедра', 'Тип_Ведомости', 'finans', 'фио', 'Дисциплина', 'status']

# Создаем трансформатор с OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Создаем конвейер для обучения
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=43))
])

# Преобразуем метки
y_train = y_train.map({2: 2, 3: 3, 4: 3, 5: 3})

# Обучаем модель
model.fit(X_train, y_train)

# Данные студента
student_data = {
    'Семестр': 2,
    'Дисциплина': 'Физика',
    'Кафедра': 'каф.Физики',
    'Тип_Ведомости': 'Экзамен',
    'Иностранец': False,
    'finans': 'Бюджет',
    'status': 'Учащийся',
    'visit': 89.727093,
    'Группа': 3182,
    'фио': 'МалышевМихаилНиколаевич, ШаталоваАлександраБорисовна'
}

# Подготавливаем данные студента для предсказания
student_df = pd.DataFrame([student_data])

# Выполняем предсказание
base_prediction = model.predict(student_df)
print(f"Base prediction: {base_prediction[0]}")

# Определяем возможные изменения для каждого признака
combinations = {
    'Семестр': [1, 2, 3],
    'Тип_Ведомости': ['Экзамен', 'Дифференцированный зачет'],
    'Иностранец': [False, True],
    'finans': ['Бюджет', 'Целевой прием', 'Контракт'],
    'visit': [0.0, 10.0, 25.0, 40.0, 51.0, 68.0, 70.0, 90.0, 100.0],
    'Группа': [3182, 3193, 3104]
}

# Изменяем каждый признак поочередно и выполняем предсказание
for feature, values in combinations.items():
    for value in values:
        modified_data = student_data.copy()
        modified_data[feature] = value

        modified_df = pd.DataFrame([modified_data])
        
        # Прогноз
        prediction = model.predict(modified_df)
        print(f"{feature} изменяем на {value} -> Ожидаемая оценка: {prediction[0]}")

Base prediction: 3
Семестр изменяем на 1 -> Ожидаемая оценка: 3
Семестр изменяем на 2 -> Ожидаемая оценка: 3
Семестр изменяем на 3 -> Ожидаемая оценка: 3
Тип_Ведомости изменяем на Экзамен -> Ожидаемая оценка: 3
Тип_Ведомости изменяем на Дифференцированный зачет -> Ожидаемая оценка: 3
Иностранец изменяем на False -> Ожидаемая оценка: 3
Иностранец изменяем на True -> Ожидаемая оценка: 3
finans изменяем на Бюджет -> Ожидаемая оценка: 3
finans изменяем на Целевой прием -> Ожидаемая оценка: 3
finans изменяем на Контракт -> Ожидаемая оценка: 3
visit изменяем на 0.0 -> Ожидаемая оценка: 2
visit изменяем на 10.0 -> Ожидаемая оценка: 2
visit изменяем на 25.0 -> Ожидаемая оценка: 2
visit изменяем на 40.0 -> Ожидаемая оценка: 2
visit изменяем на 51.0 -> Ожидаемая оценка: 3
visit изменяем на 68.0 -> Ожидаемая оценка: 3
visit изменяем на 70.0 -> Ожидаемая оценка: 2
visit изменяем на 90.0 -> Ожидаемая оценка: 3
visit изменяем на 100.0 -> Ожидаемая оценка: 3
Группа изменяем на 3182 -> Ожидаемая оценк

In [68]:
# для заданных значений

# Функция для обновления данных и предсказания
def update_prediction(sem, ved, foreign, fin, vis, group):
    modified_data = student_data.copy()
    modified_data['Семестр'] = sem
    modified_data['Тип_Ведомости'] = ved
    modified_data['Иностранец'] = foreign
    modified_data['finans'] = fin
    modified_data['visit'] = vis
    modified_data['Группа'] = group
    
    modified_df = pd.DataFrame([modified_data])
    
    prediction = model.predict(modified_df)
    print(f"Ожидаемая оценка: {prediction[0]}")

# Виджеты
sem_widget = widgets.Dropdown(
    options=[1, 2],
    description='Семестр:'
)

ved_widget = widgets.Dropdown(
    options=['Экзамен', 'Дифференцированный зачет'],
    description='Тип Ведомости:'
)

foreign_widget = widgets.Select(
    options=[False, True],
    description='Иностранец:'
)

fin_widget = widgets.Dropdown(
    options=['Бюджет', 'Целевой прием', 'Контракт'],
    description='Финансирование:'
)

vis_widget = widgets.SelectionSlider(
    options=[0.0, 10.0, 25.0, 40.0, 51.0, 68.0, 70.0, 90.0, 100.0],
    description='Посещаемость:'
)

group_widget = widgets.Dropdown(
    options=[3182, 3193, 3104, 3181],
    description='Группа:'
)

# Отображение виджетов
ui = widgets.VBox([sem_widget, ved_widget, foreign_widget, fin_widget, vis_widget, group_widget])

out = widgets.interactive_output(update_prediction, {
    'sem': sem_widget,
    'ved': ved_widget,
    'foreign': foreign_widget,
    'fin': fin_widget,
    'vis': vis_widget,
    'group': group_widget
})

display(ui, out)

Output()

In [69]:
## Для различных уникальных значени из таблиы

# Функция для обновления данных и предсказания
def update_prediction(sem, subj, kaf, ved, foreign, fin, vis, group):
    modified_data = student_data.copy()
    modified_data['Семестр'] = sem
    modified_data['Дисциплина'] = subj
    modified_data['Кафедра'] = kaf
    modified_data['Тип_Ведомости'] = ved
    modified_data['Иностранец'] = foreign
    modified_data['finans'] = fin
    modified_data['visit'] = vis
    modified_data['Группа'] = group
    
    modified_df = pd.DataFrame([modified_data])
    
    prediction = model.predict(modified_df)
    print(f"Ожидаемая оценка: {prediction[0]}")

# Виджеты
sem_widget = widgets.Dropdown(
    options=df_sorted['Семестр'].unique(),  
    description='Семестр:'
)

subj_widget = widgets.Dropdown(
    options=df_sorted['Дисциплина'].unique(),  
    description='Дисциплина:'
)

kaf_widget = widgets.Dropdown(
    options=df_sorted['Кафедра'].unique(),  
    description='Кафедра:'
)

ved_widget = widgets.Dropdown(
    options=df_sorted['Тип_Ведомости'].unique(), 
    description='Тип Ведомости:'
)

foreign_widget = widgets.Select(
    options=df_sorted['Иностранец'].unique(), 
    description='Иностранец:'
)

fin_widget = widgets.Dropdown(
    options=df_sorted['finans'].unique(), 
    description='Финансирование:'
)

vis_widget = widgets.SelectionSlider(
    options=sorted(df_sorted['visit'].unique()),
    description='Посещаемость:'
)

group_widget = widgets.Dropdown(
    options=df_sorted['Группа'].unique(), 
    description='Группа:'
)

# Отображение виджетов
ui = widgets.VBox([sem_widget, subj_widget, kaf_widget, ved_widget, foreign_widget, fin_widget, vis_widget, group_widget])

out = widgets.interactive_output(update_prediction, {
    'sem': sem_widget,
    'subj': subj_widget,
    'kaf': kaf_widget,
    'ved': ved_widget,
    'foreign': foreign_widget,
    'fin': fin_widget,
    'vis': vis_widget,
    'group': group_widget
})

display(ui, out)

Output()

In [47]:
print(prediction)

[3]


In [48]:
X

0        3
1        3
2        3
3        3
4        3
        ..
18183    3
18184    3
18185    3
18186    3
18187    3
Name: ПерваяПопытка, Length: 18188, dtype: int64

In [49]:
X_test

,Семестр,Дисциплина,Кафедра,Тип_Ведомости,Иностранец,finans,status,visit,Группа,фио
0,2,Алгебра и геометрия,каф.ВМ,Экзамен,False,Бюджет,Учащийся,0.000000,321,NaN
1,1,Алгебра и геометрия,каф.ВМ,Экзамен,False,Бюджет,Учащийся,24.448529,321,NaN
2,1,Алгебра и геометрия,каф.ВМ,Экзамен,False,Бюджет,Учащийся,72.794118,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
3,1,Алгебра и геометрия,каф.ВМ,Экзамен,False,Бюджет,Учащийся,78.860294,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
4,1,Алгебра и геометрия,каф.ВМ,Экзамен,False,Бюджет,Учащийся,24.448529,3101,"КуляхтинаОльгаЕвгеньевна, АбжандадзеЗурабЛеван..."
...,...,...,...,...,...,...,...,...,...,...
17279,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,False,Целевой прием,Учащийся,56.250000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
17280,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,False,Контракт,Учащийся,37.500000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
17281,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,False,Бюджет,Учащийся,50.000000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
17282,2,Эффективное лидерство,каф.МСК,Дифференцированный зачет,False,Контракт,Учащийся,56.250000,3691,"ЕрочкинаОльгаАлександровна, ЕрочкинаОльгаАлекс..."
